In [ ]:
# Перевод корпуса на русский язык

In [1]:
%pip install googletrans==4.0.0rc1

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
from googletrans import Translator
import pandas as pd
from tqdm import tqdm

# Инициализируем объект Translator
translator = Translator()

# Путь к файлу с исходными данными
input_file_path = '/home/jupyter/datasphere/project/zh_before.csv'

# Загрузка данных из CSV файла в DataFrame
df = pd.read_csv(input_file_path)
#df = df[4310:]

# Функция для перевода текста с разбиением на части, если он длиннее 15000 символов
def translate_text(text):
    max_len = 1500
    if len(text) <= max_len:
        try:
            translated = translator.translate(text, dest='ru')
            return translated.text
        except Exception as e:
            print(f"Ошибка при переводе текста {e}")
            return 'ooops'  # если перевести не удалось, по этой метке строка потом будет удалена
    else:
        parts = [text[i:i + max_len] for i in range(0, len(text), max_len)]
        translated_parts = []
        for part in parts:
            try:
                translated = translator.translate(part, dest='ru')
                translated_parts.append(translated.text)
            except Exception as e:
                print(f"Ошибка при переводе части текста, длина:  {e}")
                translated_parts.append('ooops')
        return ''.join(translated_parts)

# Применение tqdm к методу apply для отображения прогресса
tqdm.pandas()


# Перевод и промежуточное сохранение каждые 1000 строк
batch_size = 10
for start in tqdm(range(0, len(df), batch_size)):
    end = min(start + batch_size, len(df))
    
    def translate_or_skip(text):
        if len(text) > 10000:  # избавляемся от слишком длинных текстов
            return 'ooops'
        else:
            return translate_text(text)
            
    df_batch = df['text'][start:end].progress_apply(translate_or_skip).to_frame()
    df_batch.columns = ['translated_text']
    

    df_batch.to_csv('zh_after.csv', mode='a', header=False, index=False) # сохраняем по батчам


 30%|███       | 3/10 [00:04<00:11,  1.61s/it]

Ошибка при переводе текста Invalid input ConnectionInputs.RECV_PING in state ConnectionState.CLOSED



 30%|███       | 3/10 [00:04<00:11,  1.59s/it]

Ошибка при переводе части текста, длина:  list index out of range



 20%|██        | 2/10 [00:05<00:20,  2.54s/it]

Ошибка при переводе текста The read operation timed out



 40%|████      | 4/10 [00:09<00:17,  2.94s/it]

Ошибка при переводе текста The read operation timed out



 80%|████████  | 8/10 [00:15<00:04,  2.18s/it]